In [1]:
from time import time, sleep

from algosdk import account, encoding
from algosdk.logic import get_application_address
from auction.operations import createAuctionApp, setupAuctionApp, placeBid, closeAuction
from auction.util import (
    getBalances,
    getAppGlobalState,
    getLastBlockTimestamp,
)
from auction.testing.setup import getAlgodClient
from auction.testing.resources import (
    getTemporaryAccount,
    optInToAsset,
    createDummyAsset,
)

In [2]:
client = getAlgodClient()
print("Generating temporary accounts...")
creator = getTemporaryAccount(client)
seller = getTemporaryAccount(client)
bidder = getTemporaryAccount(client)

print("Alice (seller account):", seller.getAddress())
print("Bob (auction creator account):", creator.getAddress())
print("Carla (bidder account)", bidder.getAddress(), "\n")


Generating temporary accounts...
Alice (seller account): I2IXJCGPLT6AAMS73DQW3SRRAUFPWOFVLOQNVNIOISM26N7QZ2JSWA26AA
Bob (auction creator account): FPRFEHUTRDSIQ5UP3LPCGOEKZSABG6DFWOO3ZN5LU535G3GZYF6ZR3RUCE
Carla (bidder account) DY5ODJSZMW6SDPKJ4VGEQWIORNPIZKJZLXXS7NFW4QRLITKNEVUBHRBJIQ 



In [3]:
print("Alice is generating an example NFT...")
nftAmount = 1
nftID = createDummyAsset(client, nftAmount, seller)
print("The NFT ID is", nftID)
print("Alice's balances:", getBalances(client, seller.getAddress()), "\n")

Alice is generating an example NFT...
The NFT ID is 17
Alice's balances: {0: 99999000, 17: 1} 



In [4]:
startTime = int(time()) + 10  # start time is 10 seconds in the future
endTime = startTime + 60  # end time is 30 seconds after start
reserve = 1_000_000  # 1 Algo
increment = 100_000  # 0.1 Algo
print("Bob is creating an auction that lasts 30 seconds to auction off the NFT...")
appID = createAuctionApp(
    client=client,
    sender=creator,
    seller=seller.getAddress(),
    nftID=nftID,
    startTime=startTime,
    endTime=endTime,
    reserve=reserve,
    minBidIncrement=increment,
)
print(
    "Done. The auction app ID is",
    appID,
    "and the escrow account is",
    get_application_address(appID),
    "\n",
)

Bob is creating an auction that lasts 30 seconds to auction off the NFT...
Done. The auction app ID is 18 and the escrow account is TYLKPGKW3Q2XA3WRE4VILV3SAJXBC2OIK37RMBO52Q6K4MAPMXKVXLNFCM 



In [5]:
print("Alice is setting up and funding NFT auction...")
setupAuctionApp(
    client=client,
    appID=appID,
    funder=creator,
    nftHolder=seller,
    nftID=nftID,
    nftAmount=nftAmount,
)
print("Done\n")


Alice is setting up and funding NFT auction...
Done



In [6]:
sellerBalancesBefore = getBalances(client, seller.getAddress())
sellerAlgosBefore = sellerBalancesBefore[0]
print("Alice's balances:", sellerBalancesBefore)

_, lastRoundTime = getLastBlockTimestamp(client)
if lastRoundTime < startTime + 5:
    sleep(startTime + 5 - lastRoundTime)
actualAppBalancesBefore = getBalances(client, get_application_address(appID))
print("Auction escrow balances:", actualAppBalancesBefore, "\n")


Alice's balances: {0: 99998000, 17: 0}
Auction escrow balances: {0: 202000, 17: 1} 



In [7]:
bidAmount = reserve
bidderBalancesBefore = getBalances(client, bidder.getAddress())
bidderAlgosBefore = bidderBalancesBefore[0]
print("Carla wants to bid on NFT, her balances:", bidderBalancesBefore)
print("Carla is placing bid for", bidAmount, "microAlgos")

placeBid(client=client, appID=appID, bidder=bidder, bidAmount=bidAmount)

print("Carla is opting into NFT with ID", nftID)

optInToAsset(client, nftID, bidder)

print("Done\n")

Carla wants to bid on NFT, her balances: {0: 100000000}
Carla is placing bid for 1000000 microAlgos
so far so good!!........


AlgodHTTPError: TransactionPool.Remember: transaction M6SKRBFLI5SVRYGPGUEWWGB3ONXDYRALGVFH4JXMSS5NIH65DQXQ: logic eval error: assert failed pc=335. Details: pc=335, opcodes=>=
&&
assert


In [ ]:
_, lastRoundTime = getLastBlockTimestamp(client)
if lastRoundTime < endTime + 5:
    waitTime = endTime + 5 - lastRoundTime
    print("Waiting {} seconds for the auction to finish\n".format(waitTime))
    sleep(waitTime)

print("Alice is closing out the auction\n")
closeAuction(client, appID, seller)

In [ ]:
actualAppBalances = getBalances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The auction escrow now holds the following:", actualAppBalances)
assert actualAppBalances == expectedAppBalances

bidderNftBalance = getBalances(client, bidder.getAddress())[nftID]
assert bidderNftBalance == nftAmount

In [ ]:
actualSellerBalances = getBalances(client, seller.getAddress())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")
actualBidderBalances = getBalances(client, bidder.getAddress())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidAmount - 1_000
assert actualSellerBalances[nftID] == 0